<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-3/blob/main/ML_Zoomcamp_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 20/09/2022**

###3.1 Churn prediction project

In [74]:
# # Import libraries for the project
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [75]:
# Locate dataset for the project
data = 'https://github.com/alexeygrigorev/mlbookcamp-code/blob/fc43fe928f1b6d0a5143ce89110ddc135b1221e8/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [76]:
# Download dataset and store it locally with different file name
!wget $data -O data.csv

--2022-09-20 15:03:40--  https://github.com/alexeygrigorev/mlbookcamp-code/blob/fc43fe928f1b6d0a5143ce89110ddc135b1221e8/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data.csv’

data.csv                [   <=>              ]   2.62M  5.23MB/s    in 0.5s    

2022-09-20 15:03:41 (5.23 MB/s) - ‘data.csv’ saved [2742972]



In [77]:
# Read the dataset, store it and print the first rows
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [78]:
# Transpose the dataframe for better visualization of the column names
# using the transpose the rows become columns
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


###3.2 Data preparation

In [79]:
# after we print the data we check for inconsistency e.g. names of the columns
# on this data we have underscores, spaces between the names
# what we need to do is cleaning to make it consistent e.g. make it lower case
# everywhere and replace spaces with underscores

# Make the categorical data of the dataframe consistent 
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = strings = list(df.dtypes[df.dtypes == 'object'].index)

for col in categorical_columns:
  df[col] = df[col].str.lower().str.replace(' ', '_')

In [80]:
# Check the categorical data of the dataframe is now consistent 
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [81]:
# Print the data types of the dataframe
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [82]:
# Convert a serie of the dataframe to a number
tc = pd.to_numeric(df.totalcharges, errors='coerce')
tc
# note: using the parameter errors it will avoid an error called ValueError: Unable to parse string "_"

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: totalcharges, Length: 7043, dtype: float64

In [83]:
# Find the missing values of the total charges serie of the dataframe
tc.isnull().sum()

11

In [84]:
# Check the missing values of the total charges serie of the dataframe
df[tc.isnull()]

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
488,4472-lvygi,female,0,yes,yes,0,no,no_phone_service,dsl,yes,...,yes,yes,yes,no,two_year,yes,bank_transfer_(automatic),52.55,_,no
753,3115-czmzd,male,0,no,yes,0,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.25,_,no
936,5709-lvoeq,female,0,yes,yes,0,yes,no,dsl,yes,...,yes,no,yes,yes,two_year,no,mailed_check,80.85,_,no
1082,4367-nuyao,male,0,yes,yes,0,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,25.75,_,no
1340,1371-dwpaz,female,0,yes,yes,0,no,no_phone_service,dsl,yes,...,yes,yes,yes,no,two_year,no,credit_card_(automatic),56.05,_,no
3331,7644-omvmy,male,0,yes,yes,0,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.85,_,no
3826,3213-vvolg,male,0,yes,yes,0,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,25.35,_,no
4380,2520-sgtta,female,0,yes,yes,0,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.00,_,no
5218,2923-arzlg,male,0,yes,yes,0,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,one_year,yes,mailed_check,19.70,_,no
6670,4075-wkniu,female,0,yes,yes,0,yes,yes,dsl,no,...,yes,yes,yes,no,two_year,no,mailed_check,73.35,_,no


In [85]:
# Check the missing values of the total charges serie of the dataframe
# select only two columns for better visualization
df[tc.isnull()][['customerid','totalcharges']]

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [86]:
# Fill the missing values of the total charges serie with zeros
df.totalcharges.fillna(0)

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: totalcharges, Length: 7043, dtype: object

In [87]:
# Print the first five churn variable of the dataframe
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [88]:
# Replace the churn data with numbers
(df.churn == 'yes').astype(int).head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

In [89]:
# Update the churn data with numbers
df.churn = (df.churn == 'yes').astype(int)

###3.3 Setting up the validation framework

In [90]:
# Import the library sklearn with the project
from sklearn.model_selection import train_test_split

In [91]:
# to find the documentation for this function use ? 
train_test_split?
# here we are interrested in the test_size parameter and it says how large the dataset should be

In [92]:
# Define the sizes of the datasets with 20% and use random state so the results are reproducible
# the full train dataset has 80% and the test 20%
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [93]:
# Print the sizes of the dataframes full train and test
len(df_full_train), len(df_test)

(5634, 1409)

In [94]:
# Split the full train dataframe in train and validation dataframes
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
# note: the validation size is 25% because its 20% of the 80% of the full train dataframe

In [95]:
# Print the sizes of the dataframes train, validation and test
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [96]:
# Reset the indexes of train, validation and test dataframes
df_train = df_train.reset_index(drop=True)
df_val   = df_val.reset_index(drop=True)
df_test  = df_test.reset_index(drop=True)

In [97]:
# Get the churn variables from train, validation and test dataframes
y_train = df_train.churn.values
y_val   = df_val.churn.values
y_test  = df_test.churn.values

In [98]:
# Delete the serie churn from the dataframes: train, validation and test
del df_train['churn']
del df_val['churn']
del df_test['churn']

# note: we didn't delete the churn variable from the full train dataframe 
# and the reason it will be explained in the next lesson

###3.4 EDA (Exploratory Data Analysis)